## Homework 9 Question 1
##### 2022-07-20

In [16]:
import warnings
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
warnings.filterwarnings("ignore")

### Part A

In [2]:
column_names = pd.read_csv("MovieReviewMat.csv", index_col=0, header=0, nrows=0).columns
genres = pd.read_csv("MovieReviewMat.csv", index_col=0, header=0, nrows=1).transpose()
genres.columns = ['genres']
x = pd.read_csv("MovieReviewMat.csv", index_col=0, skiprows=2, names=column_names)

print(x.max().max())
print(x.min().min())
print(np.nansum(x) / np.count_nonzero(x.fillna(0)))

5.0
0.5
3.297080189685473


The highest score is a __5.0__.

The lowest score is a __0.5__.

The mean score is __3.30__.

### Part B

In [3]:
x.iloc[1460, :][x.iloc[1460, :] == 5.0].to_frame().join(genres)

,1462,genres
Taxi Driver (1976),5.0,Crime|Drama|Thriller
Rob Roy (1995),5.0,Action|Drama|Romance|War
"Little Princess, A (1995)",5.0,Children|Drama
William Shakespeare's Romeo + Juliet (1996),5.0,Drama|Romance
"Quiet Man, The (1952)",5.0,Drama|Romance
Raging Bull (1980),5.0,Drama
Pink Floyd: The Wall (1982),5.0,Drama|Musical
Field of Dreams (1989),5.0,Children|Drama|Fantasy
"Man Who Would Be King, The (1975)",5.0,Adventure|Drama
Playing by Heart (1998),5.0,Drama|Romance


Reviewer 1462 tends to give __Dramas__ a score of 5.0.

In [4]:
x.iloc[1460, :][x.iloc[1460, :] == 5.0] \
    .to_frame() \
    .merge(x.iloc[43, :].to_frame(), left_index=True, right_index=True) \
    .join(genres)

,1462,45,genres
Taxi Driver (1976),5.0,1.5,Crime|Drama|Thriller
Rob Roy (1995),5.0,NaN,Action|Drama|Romance|War
"Little Princess, A (1995)",5.0,2.5,Children|Drama
William Shakespeare's Romeo + Juliet (1996),5.0,2.5,Drama|Romance
"Quiet Man, The (1952)",5.0,2.5,Drama|Romance
Raging Bull (1980),5.0,2.5,Drama
Pink Floyd: The Wall (1982),5.0,3.5,Drama|Musical
Field of Dreams (1989),5.0,NaN,Children|Drama|Fantasy
"Man Who Would Be King, The (1975)",5.0,2.0,Adventure|Drama
Playing by Heart (1998),5.0,3.0,Drama|Romance


Reviewer 45 tends to give scores of 2.5 to the __Dramas__ that reviewer 1462 likes.

In [5]:
x.iloc[43, :][x.iloc[43, :] == 5.0].to_frame().join(genres)

,45,genres
Jack (1996),5.0,Comedy|Drama
"Sting, The (1973)",5.0,Comedy|Crime
Groundhog Day (1993),5.0,Comedy|Fantasy|Romance
Howard the Duck (1986),5.0,Adventure|Comedy|Sci-Fi
Anywhere But Here (1999),5.0,Comedy|Drama


Reviewer 45 tends to give __Comedies__ a score of 5.0.

Reviewer 1462 game __Taxi Driver (1976)__ and __Rob Roy (1995)__ the highest score.

Reviewer 45 game __Jack (1996)__ and __Sting, The (1973)__ the highest score.

### Part C

Given that scores of 5.0 appear to be given out by reviewers only to movies in their favorite genres, I estimate there will be a type of individual for each genre. So the number of types of individuals would be equal to the number of genres. I am able to think of roughly 8 general genre categories: Action, Adventure, Comedy, Romance, Drama, Horror, Documentary, and Children. For this reason, we will use $k=8$.

In [6]:
k = 8

### Part D

In [7]:
U, S, V = np.linalg.svd(x.fillna(0).to_numpy())
x_completed = np.dot(U[:, :k], np.dot(np.diag(S[:k]), V[:k, :]))

def calculate_rmse(true, recon):
    true_without_nans = true.fillna(0).to_numpy()
    diff = true_without_nans[true_without_nans != 0] - recon[true_without_nans != 0]
    return np.sqrt(np.sum((diff) ** 2) / np.count_nonzero(true_without_nans))

print(x_completed[43, 1])
print(calculate_rmse(x, x_completed))

1.5883529135408097
1.8105889042288899


Reviewer 45 is predicted to give _Jumanji (1995)_ a score of $1.59$.

The average difference between the true, non-missing values and the predictions is $1.81$.

### Part E

In [9]:
x_column_mean = x.copy()
column_means = np.nanmean(x_column_mean, axis=0)
for i in range(x_column_mean.shape[1]):
    x_column_mean.iloc[:, i] = x_column_mean.iloc[:, i].fillna(column_means[i])

U, S, V = np.linalg.svd(x_column_mean.to_numpy())
x_column_completed = np.dot(U[:, :k], np.dot(np.diag(S[:k]), V[:k, :]))

print(x_column_completed[43, 1])
print(calculate_rmse(x, x_column_completed))

2.4512508755173434
0.7956369720803059


Reviewer 45 is predicted to give _Jumanji (1995)_ a score of $2.45$.

The average difference between the true, non-missing values and the predictions is $0.80$.

### Part F

In [7]:
x_row_mean = x.copy()
row_means = np.nanmean(x_row_mean, axis=1)
for i in range(x_row_mean.shape[0]):
    x_row_mean.iloc[i, :] = x_row_mean.iloc[i, :].fillna(row_means[i])

U, S, V = np.linalg.svd(x_row_mean.to_numpy())
x_row_completed = np.dot(U[:, :k], np.dot(np.diag(S[:k]), V[:k, :]))

print(x_row_completed[43, 1])
print(calculate_rmse(x, x_row_completed))

2.440392257114829
0.8104082321508662


Reviewer 45 is predicted to give _Jumanji (1995)_ a score of $2.44$.

The average difference between the true, non-missing values and the predictions is $0.81$.

### Part G

Replacing the missing data with zeroes is a naive approach. The resulting model learns that reviewers give scores of zero instead of what they might actually give. This is not a good approach for this data as can be seen by the results. The predicted score is much lower than the other two; it is pulled down by the averaging from the zeros. Thus, the root mean squared error is much worse.

The column mean approach is much more effective. It assumes that other reviewers will give the same movie similar scores. This gives a much better resulting root mean squared error.

The row mean approach is also much more effective than the naive zero approach. It is about as effective as the column mean by the result of the root mean squared error. It assumes that other movies will be given a similar score by the same reviewer.

Another possible method for imputing the missing values is the mean of the entire matrix. This would likely work better than zeros, but would not be as effective as imputing across columns or rows as it would pull the entire prediction matrix toward the total mean. This does not have quite as much granularity as dividing the data up by column or row.

### Part H

In [31]:
nmf = NMF(n_components=k, random_state=0)
W = nmf.fit_transform(x_column_mean.to_numpy())
x_column_mean_nmf_completed = nmf.inverse_transform(W)

print(x_column_mean_nmf_completed[43, 1])
print(calculate_rmse(x, x_column_mean_nmf_completed))

2.390068087243282
0.8058970682676835


Reviewer 45 is predicted to give _Jumanji (1995)_ a score of $2.39$.

The average difference between the true, non-missing values and the predictions is $0.81$.

The NMF model has nearly the same root mean squared error as the column mean and row mean SVD. The prediction is also nearly the same.

### Part I

SVD vectors are always global; you are guaranteed to find global optima, as they are simply linear combinations. On top of this, SVD is a _special case_ non-convex problem where we do in fact have efficient algorithms, unlike most non-convex problems. You also need to have more understanding and context on the data to effectively execute NMF; it is possible to get stuck in local optima. You need conditions based on all this to prevent getting stuck in local optima. For an SVD, this isn't necessary. However, if it is possible to condition correctly, NMF is much more easily interpretable; the results given are much clearer.

### Part J

In [63]:
nmf = NMF(n_components=5, random_state=0)
W = nmf.fit_transform(x_column_mean.to_numpy())
x_column_mean_nmf_completed = nmf.inverse_transform(W)

movies_not_watched = x.copy().iloc[1460, :].fillna(0)
movies_not_watched = movies_not_watched[movies_not_watched == 0].index

recs = pd.DataFrame(x_column_mean_nmf_completed)
recs.columns = x.columns
recs = recs.loc[1460, movies_not_watched]
recs.sort_values(ascending=False).to_frame().join(genres).head(3)

,1460,genres
Before Sunset (2004),4.054311,Drama|Romance
K-PAX (2001),3.971173,Drama|Fantasy|Mystery|Sci-Fi
The Falcon and the Snowman (1985),3.933291,Crime|Drama|Thriller


We recommend _Before Sunset_, _K-PAX_, and _The Falcon and the Snowman_ for reviewer 1462. After running an NMF, these movies have the highest predicted rating by him from the list of movies he has not yet reviewed. Given that from Part B we concluded that reviewer 1462 prefers __Dramas__ and sometimes __Romance__, we expect these to be quite good recommendations. We expect this reviewer to enjoy these movies.

### Part K

In [62]:
k = 5
L = 100
x_completed = x.fillna(0).to_numpy()
missing_indices = np.where(x_completed == 0)

for i in range(100):
    U, S, V = np.linalg.svd(x_completed)
    S[S <= L] = 0
    S[S > L] = S[S > L] - L
    x_recon = np.dot(U[:, :k], np.dot(np.diag(S[:k]), V[:k, :]))
    x_completed[missing_indices] = x_recon[missing_indices]

movies_not_watched = x.copy().iloc[43, :].fillna(0)
movies_not_watched = movies_not_watched[movies_not_watched == 0].index

recs = pd.DataFrame(x_completed)
recs.columns = x.columns
recs = recs.loc[43, movies_not_watched]
recs.sort_values(ascending=False).to_frame().join(genres).head(3)

,43,genres
"Ghost and Mrs. Muir, The (1947)",3.585019,Drama|Fantasy|Romance
2001: A Space Odyssey (1968),3.550058,Adventure|Drama|Sci-Fi
M (1931),3.471073,Crime|Film-Noir|Thriller


We recommend _The Ghost and Mrs. Muir_, _2001: A Space Odyssey_, and _M_ for reviewer 45. After running a soft impute, these movies have the highest predicted rating by him from the list of movies he has not yet reviewed. Given that from Part B we concluded that reviewer 45 prefers __Comedies__, we expect these to be very bad recommendations. We don't expect this reviewer to enjoy these movies.